In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


/Users/mustainbillah/Projects/sentiment-analysis/venv-sentiment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set the tracking URI to the local mlruns directory
mlflow.set_tracking_uri("file:///Users/mustainbillah/Projects/sentiment-analysis/mlruns")

# Create and set the experiment
experiment_name = "04_ML_Algorithm_Comparison_with_HPT"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
except:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///Users/mustainbillah/Projects/sentiment-analysis/mlruns/658432777505770578', creation_time=1748791728665, experiment_id='658432777505770578', last_update_time=1748791728665, lifecycle_stage='active', name='04_ML_Algorithm_Comparison_with_HPT', tags={}>

In [3]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [4]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Multinomial Naive Bayes

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup
    model = MultinomialNB(alpha=alpha)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "MultinomialNB"
    log_mlflow("MultinomialNB", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()


[I 2025-06-02 11:10:07,784] A new study created in memory with name: no-name-011c7830-9693-4ef4-adac-785d13cdc685
[I 2025-06-02 11:10:07,801] Trial 0 finished with value: 0.6673007820756711 and parameters: {'alpha': 0.0002521550256140671}. Best is trial 0 with value: 0.6673007820756711.
[I 2025-06-02 11:10:07,807] Trial 1 finished with value: 0.6673007820756711 and parameters: {'alpha': 0.0019470751351494937}. Best is trial 0 with value: 0.6673007820756711.
[I 2025-06-02 11:10:07,812] Trial 2 finished with value: 0.6673007820756711 and parameters: {'alpha': 0.005328659722402065}. Best is trial 0 with value: 0.6673007820756711.
[I 2025-06-02 11:10:07,818] Trial 3 finished with value: 0.6673007820756711 and parameters: {'alpha': 0.0007050131480746919}. Best is trial 0 with value: 0.6673007820756711.
[I 2025-06-02 11:10:07,825] Trial 4 finished with value: 0.6673007820756711 and parameters: {'alpha': 0.00013116074449827115}. Best is trial 0 with value: 0.6673007820756711.
[I 2025-06-02 11